In [1]:
import numpy as np
import pandas as pd
import itertools
import seaborn as sns
from iit_tpm import make_tpm
from grn_tpm import iit_tpm_cal

# 不同环境状态下系统演化序列比较

In [2]:
w1 = {
    'w1': 0.7,
    'w2': 0.2,
    'w3': -0.8,
    'w4': 0.5,
    'w5': -0.03,
    'w6': -0.02
}

bnet = {
    ('A', 'B'): 'w1',
    ('B', 'A'): 'w1',
    ('A', 'C'): 'w2',
    ('C', 'A'): 'w2',
    ('B', 'C'): 'w3',
    ('C', 'B'): 'w3',
    ('A', 'A'): 'w4',
    ('B', 'B'): 'w4',
    ('C', 'C'): 'w4',
    ('E1', 'A'): 'w5',
    ('E1', 'B'): 'w5',
    ('E1', 'C'): 'w5',
    ('E2', 'A'): 'w6',
    ('E2', 'B'): 'w6',
    ('E2', 'C'): 'w6',
}

In [3]:
tpm, tpm_v = make_tpm(bnet, w1, k=1)
un, en, iit_tpm_cal(tpm_v, mech_size=3, en_size=2)

un:  0.08329894433977869
un_en:  0.16450925694093038
syn:  0.0031829001159548315


(0.08329894433977869,
 0.16450925694093038,
 0.0031829001159548315,
 {'00': array([[0.18367033, 0.04761474, 0.18367033, 0.04761474, 0.21339448,
          0.05532044, 0.21339448, 0.05532044],
         [0.10731125, 0.07562096, 0.10731125, 0.07562096, 0.18599754,
          0.13107025, 0.18599754, 0.13107025],
         [0.19190522, 0.04685237, 0.19190522, 0.04685237, 0.20997775,
          0.05126466, 0.20997775, 0.05126466],
         [0.11417471, 0.07577207, 0.11417471, 0.07577207, 0.18636923,
          0.12368399, 0.18636923, 0.12368399],
         [0.08671016, 0.03353438, 0.08671016, 0.03353438, 0.27384743,
          0.10590803, 0.27384743, 0.10590803],
         [0.04267751, 0.04486563, 0.04267751, 0.04486563, 0.2010738 ,
          0.21138307, 0.2010738 , 0.21138307],
         [0.09222063, 0.03358851, 0.09222063, 0.03358851, 0.27428944,
          0.09990142, 0.27428944, 0.09990142],
         [0.04621033, 0.04575053, 0.04621033, 0.04575053, 0.20503966,
          0.20299948, 0.20503966, 0.2